In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("data/datingTestSet2.txt",  sep="\t", header=None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
0    1000 non-null int64
1    1000 non-null float64
2    1000 non-null float64
3    1000 non-null int64
dtypes: float64(2), int64(2)
memory usage: 31.3 KB


In [3]:
df.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,33635.421000,6.559961,0.832073,1.985000
std,21957.006833,4.243618,0.497239,0.818196
min,0.000000,0.000000,0.001156,1.000000
25%,13796.000000,2.933963,0.408995,1.000000
50%,31669.000000,6.595204,0.809420,2.000000
75%,47716.250000,10.056500,1.272847,3.000000
max,91273.000000,20.919349,1.695517,3.000000


In [4]:
# DataFrame转ndarray
x = df.iloc[:,0:-1].values
x.shape

(1000, 3)

In [5]:
y = df.iloc[:,-1].values
y.shape

(1000,)

In [6]:
numTest = int(x.shape[0] * 0.1)
numTrain = x.shape[0] - numTest
numTrain

900

In [7]:
x_train, x_test = x[:numTrain,:], x[numTrain:,:]
y_train, y_test = y[:numTrain], y[numTrain:]

In [8]:
from sklearn import neighbors 
from sklearn.preprocessing import MinMaxScaler

In [9]:
# 归一化
mm = MinMaxScaler()
mm.fit(x_train)
x_mm_train = mm.transform(x_train)

In [10]:
x_mm_train

array([[0.44832535, 0.39805139, 0.56233353],
       [0.15873259, 0.34195467, 0.98724416],
       [0.28542943, 0.06892523, 0.47449629],
       ...,
       [0.05496697, 0.35915807, 0.08037956],
       [0.11110624, 0.39393162, 0.05818064],
       [0.38970999, 0.6985302 , 0.73551858]])

In [11]:
x_mm_test = mm.transform(x_test)
x_mm_test[0:5]

array([[0.51376639, 0.17031964, 0.26218144],
       [0.08959933, 0.15442555, 0.78527657],
       [0.6111665 , 0.17268931, 0.91524474],
       [0.01257765, 0.        , 0.19547723],
       [0.11024071, 0.18792645, 0.28708227]])

In [12]:
knn = neighbors.KNeighborsClassifier(n_neighbors=5)

In [13]:
#用KNN的分类器进行建模，这里利用的默认的参数
knn.fit(x_mm_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [14]:
# 预测
predictedLabel = knn.predict(x_mm_test)

In [15]:
predictedLabel == y_test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [16]:
predictedLabel[predictedLabel != y_test]

array([1, 2, 2, 1, 2])

In [17]:
# 100个，预测对了95个。
knn.score(x_mm_test, y_test)

0.95

为啥这个比我手写的准确呢，发现我手写的是统一对x归一化，然后才切分训练集和测试集，用sklearn也统一进行归一化和我的结果是一样的。
归一化的训练是怎么影响的的?如果切分的数据最大值最小值一样的话，怎么归一化就是都是没有区别的。

In [18]:
# 归一化
mm = MinMaxScaler()
mm.fit(x)
x_norm = mm.transform(x)

In [19]:
numTest = int(x_norm.shape[0] * 0.1)
numTrain = x_norm.shape[0] - numTest
numTrain

900

In [20]:
# 注意，就是在这个写错了(没有用归一化的数据)，造成第一次写的与用sklearn测试的相差加到
x_train, x_test = x[:numTrain,:], x[numTrain:,:]
y_train, y_test = y[:numTrain], y[numTrain:]

In [21]:
knn = neighbors.KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)
rst_ys = knn.predict(x_test)

In [22]:
y_test[rst_ys != y_test]

array([1, 1, 3, 3, 1, 1, 3, 3, 2, 1, 3, 3, 2, 3, 2, 2, 3, 3])

In [23]:
y_test[rst_ys != y_test].shape

(18,)

这个和我手写的结果是一样的，看看数据里有啥不一样的数据。

In [24]:
df.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,33635.421000,6.559961,0.832073,1.985000
std,21957.006833,4.243618,0.497239,0.818196
min,0.000000,0.000000,0.001156,1.000000
25%,13796.000000,2.933963,0.408995,1.000000
50%,31669.000000,6.595204,0.809420,2.000000
75%,47716.250000,10.056500,1.272847,3.000000
max,91273.000000,20.919349,1.695517,3.000000


In [25]:
df[:900].describe()

,0,1,2,3
count,900.000000,900.000000,900.000000,900.000000
mean,33735.318889,6.579974,0.835152,1.983333
std,21818.600701,4.217952,0.498852,0.822886
min,0.000000,0.000000,0.001156,1.000000
25%,14179.000000,2.986646,0.408995,1.000000
50%,32040.500000,6.612266,0.815150,2.000000
75%,47524.500000,10.056500,1.272847,3.000000
max,91273.000000,20.919349,1.695517,3.000000


基本是一样的，再仔细观察才发现手写时使用的是未归一化的数据。卧槽！